In [1]:
import sys

sys.version

'2.7.18 |Anaconda, Inc.| (default, Apr 23 2020, 22:42:48) \n[GCC 7.3.0]'

In [ ]:
from pyswip import Prolog

prolog = Prolog()
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")

for soln in prolog.query("father(X,Y)"):
    print(soln["X"], "is the father of", soln["Y"])

# michael is the father of john
# michael is the father of gina

In [1]:
from pyswip import *

assertz = Functor("assertz", 1)
father = Functor("father", 2)
call(assertz(father("a","b")))

#q = Query(father("a","b"))
#print q.nextSolution()

1

In [ ]:
Query(father("a","b"))

In [ ]:
list(prolog.query("father(michael,X)"))